In [1]:
import pandas as pd
import numpy as np
from typing import TypeAlias

pd.options.display.precision = 4
import warnings

from sklearn.exceptions import ConvergenceWarning
from statsmodels.tsa.stattools import ValueWarning

warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore',  category=ConvergenceWarning)

true_date_format = "%y/%m/%d"

ndarrry : TypeAlias = np.ndarray
DataFrame : TypeAlias = pd.DataFrame
Series : TypeAlias = pd.Series

# USD DataFrame

In [2]:
def create_df(
        filepath: str
        ):
    df= pd.read_csv(filepath)


    df.columns = [
        "Open",
        "Low",
        "High",
        "Close",
        "change",
        "change%",
        "date[gregory]",
        "date[jalali]",
    ]
    true_hirarchy = [
        "date[gregory]",
        "date[jalali]",
        "Open",
        "Low",
        "High",
        "Close",
        "change",
        "change%",
    ]
    pd.to_datetime(df['date[gregory]'], format="ISO8601",  yearfirst=True)

    df = df[true_hirarchy]
    df.set_index('date[gregory]', inplace=True)

    df.drop("change%",axis=1, inplace=True)

    for col in df.columns[1:]:
        # print(col)
        if col == "change": 
            continue
        else:
            df[col] = df[col].map(lambda x : x.replace(",", "")).astype(np.float32)
    df.sort_index(inplace=True)
    return df[[
        "date[jalali]",
        "Open",
        "Low",
        "High",
        "Close",]]



In [3]:
usd = create_df('dol.csv')
gold = create_df('gold.csv')
uae = create_df("derham.csv")

In [4]:
usd.head()

,date[jalali],Open,Low,High,Close
date[gregory],,,,,
2023/05/30,1402/03/09,509400.0,508300.0,511600.0,509580.0
2023/05/31,1402/03/10,513110.0,511300.0,513600.0,513330.0
2023/06/01,1402/03/11,510890.0,510890.0,510890.0,510890.0
2023/06/03,1402/03/13,512090.0,505800.0,512100.0,509250.0
2023/06/06,1402/03/16,508560.0,496110.0,508590.0,496350.0


In [5]:
gold.head()

,date[jalali],Open,Low,High,Close
date[gregory],,,,,
2023/05/30,1402/03/09,2.4966e+07,2.4488e+07,2.5144e+07,2.4950e+07
2023/05/31,1402/03/10,2.4959e+07,2.4703e+07,2.5097e+07,2.5040e+07
2023/06/01,1402/03/11,2.5040e+07,2.5040e+07,2.5040e+07,2.5040e+07
2023/06/03,1402/03/13,2.4647e+07,2.4647e+07,2.4647e+07,2.4647e+07
2023/06/06,1402/03/16,2.4585e+07,2.3839e+07,2.4590e+07,2.3869e+07


In [6]:
uae.head()

,date[jalali],Open,Low,High,Close
date[gregory],,,,,
2023/06/06,1402/03/16,138890.0,135100.0,138890.0,135120.0
2023/06/07,1402/03/17,135190.0,133780.0,135780.0,134160.0
2023/06/08,1402/03/18,134160.0,134160.0,134160.0,134160.0
2023/06/10,1402/03/20,134370.0,127680.0,134370.0,127970.0
2023/06/11,1402/03/21,128940.0,128010.0,131570.0,131180.0


In [7]:
for data in [usd, gold, uae]:
    print(data.shape)

(720, 5)
(720, 5)
(720, 5)


# Combination

In [8]:
def feature_extracion(dfs : list[DataFrame], new_df_col_name: list[str]):
    assert len(dfs) == len(new_df_col_name), f"there should be enough name for dataframes, # dataframes are {len(dfs)}, there are {len(new_df_col_name)} names!"
    # draf = pd.DataFrame(index = dfs[0].index)
    for name, data in zip(new_df_col_name, dfs):
        data[name] = data['Close']
    
    df = pd.concat(dfs, axis=1)
    # print(df)
    return df [new_df_col_name]


In [9]:
df = feature_extracion([usd, gold, uae], ['usd', "gold", "uae"])

df.head()

,usd,gold,uae
date[gregory],,,
2023/05/30,509580.0,2.4950e+07,NaN
2023/05/31,513330.0,2.5040e+07,NaN
2023/06/01,510890.0,2.5040e+07,NaN
2023/06/03,509250.0,2.4647e+07,NaN
2023/06/06,496350.0,2.3869e+07,135120.0


In [10]:
df.tail()

,usd,gold,uae
date[gregory],,,
2025/12/11,NaN,NaN,342460.0
2025/12/13,NaN,NaN,351190.0
2025/12/14,NaN,NaN,353180.0
2025/12/15,NaN,NaN,355070.0
2025/12/16,NaN,NaN,353770.0


In [11]:
df.dropna(inplace=True)
df

,usd,gold,uae
date[gregory],,,
2023/06/06,496350.0,2.3869e+07,135120.0
2023/06/07,492900.0,2.3486e+07,134160.0
2023/06/08,490840.0,2.3486e+07,134160.0
2023/06/10,470090.0,2.2472e+07,127970.0
2023/06/11,481970.0,2.2768e+07,131180.0
...,...,...,...
2025/12/04,1197050.0,1.2248e+08,326040.0
2025/12/06,1219950.0,1.2450e+08,332180.0
2025/12/07,1240050.0,1.2706e+08,337620.0


# Check for Correlation

In [12]:
def add_return(df: DataFrame):
    for col in df.columns:
        df[f'{col} return'] = df[col].pct_change()
    df.dropna(inplace=True)
    return df

In [13]:
df = add_return(df)
df

,usd,gold,uae,usd return,gold return,uae return
date[gregory],,,,,,
2023/06/07,492900.0,2.3486e+07,134160.0,-0.0070,-0.0160,-0.0071
2023/06/08,490840.0,2.3486e+07,134160.0,-0.0042,0.0000,0.0000
2023/06/10,470090.0,2.2472e+07,127970.0,-0.0423,-0.0432,-0.0461
2023/06/11,481970.0,2.2768e+07,131180.0,0.0253,0.0132,0.0251
2023/06/12,499100.0,2.3465e+07,135900.0,0.0355,0.0306,0.0360
...,...,...,...,...,...,...
2025/12/04,1197050.0,1.2248e+08,326040.0,-0.0062,-0.0038,-0.0059
2025/12/06,1219950.0,1.2450e+08,332180.0,0.0191,0.0165,0.0188
2025/12/07,1240050.0,1.2706e+08,337620.0,0.0165,0.0206,0.0164


In [14]:
df.corr(method="pearson")

,usd,gold,uae,usd return,gold return,uae return
usd,1.0000,0.9846,0.9992,0.0752,0.0870,0.0734
gold,0.9846,1.0000,0.9829,0.0597,0.0853,0.0655
uae,0.9992,0.9829,1.0000,0.0694,0.0889,0.0772
usd return,0.0752,0.0597,0.0694,1.0000,0.6082,0.7706
gold return,0.0870,0.0853,0.0889,0.6082,1.0000,0.7667
uae return,0.0734,0.0655,0.0772,0.7706,0.7667,1.0000


In [15]:
df.corr(method="spearman")


,usd,gold,uae,usd return,gold return,uae return
usd,1.0000,0.9822,0.9947,0.1217,0.1030,0.0987
gold,0.9822,1.0000,0.9840,0.0966,0.1133,0.0947
uae,0.9947,0.9840,1.0000,0.1074,0.1071,0.1045
usd return,0.1217,0.0966,0.1074,1.0000,0.6439,0.8328
gold return,0.1030,0.1133,0.1071,0.6439,1.0000,0.7080
uae return,0.0987,0.0947,0.1045,0.8328,0.7080,1.0000


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 713 entries, 2023/06/07 to 2025/12/09
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   usd          713 non-null    float32
 1   gold         713 non-null    float32
 2   uae          713 non-null    float32
 3   usd return   713 non-null    float32
 4   gold return  713 non-null    float32
 5   uae return   713 non-null    float32
dtypes: float32(6)
memory usage: 22.3+ KB


In [17]:
df['gold return']

date[gregory]
2023/06/07   -0.0160
2023/06/08    0.0000
2023/06/10   -0.0432
2023/06/11    0.0132
2023/06/12    0.0306
               ...  
2025/12/04   -0.0038
2025/12/06    0.0165
2025/12/07    0.0206
2025/12/08    0.0005
2025/12/09   -0.0026
Name: gold return, Length: 713, dtype: float32

In [18]:
np.log(df['gold return'].abs())

date[gregory]
2023/06/07   -4.1323
2023/06/08      -inf
2023/06/10   -3.1425
2023/06/11   -4.3297
2023/06/12   -3.4863
               ...  
2025/12/04   -5.5605
2025/12/06   -4.1068
2025/12/07   -3.8808
2025/12/08   -7.5478
2025/12/09   -5.9388
Name: gold return, Length: 713, dtype: float32

# Examine causality between two price series 
## Output: 
    Granger (linear) based detection and MLP based nonlinear test

In [19]:


from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.api import VAR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from scipy.stats import ttest_rel

# ---------- General settings ----------
SIGNIFICANCE = 0.05
MAX_LAG = 1   # The maximum lag we consider in model selection
MLP_HIDDEN = (128,)
MLP_EPOCHS = 500
TEST_SIZE_RATIO = 0.3
TS_SPLITS = 5   # For cross-validation

# ---------- Auxiliary functions ----------
def log_returns(series : Series):
    if np.any(series <0):
        return safe_log_returns(series)
    else:
        return np.log(series).diff().dropna() # type: ignore

def safe_log_returns(series: pd.Series):
    series = series[series > 0]  # Only positive values
    return np.log(series).diff().dropna() # type: ignore


def make_stationary(series: Series, max_diff=2):
    # Apply difference until ADF stops (or reaches max_diff)
    s = series.copy().dropna()
    d = 0
    pvalue = adfuller(s)[1]
    while pvalue > 0.05 and d < max_diff:
        s = s.diff().dropna()
        d += 1
        pvalue = adfuller(s)[1]
    return s, d, pvalue

def select_var_lag(df, maxlags=10, ic='aic'):
    model = VAR(df)
    res = model.select_order(maxlags)
    lag = getattr(res, ic)
    if np.isnan(lag):
        # fallback: choose 1
        return 1
    return int(lag)

# ---------- 1) Granger test (linear) ----------
def granger_test_matrix(x, y, maxlag=10, significance=0.05):
    """
    x, y: pandas Series (stationary). بررسی می‌کنیم x -> y و y -> x با انتخاب lag مناسب.
    برمی‌گرداند: dict با keys 'x_causes_y' و 'y_causes_x' شامل (p-value, chosen_lag, boolean)
    """
    df = pd.concat([x, y], axis=1).dropna()
    df.columns = ['x', 'y']
    chosen_lag = select_var_lag(df, maxlags=min(maxlag, len(df)//5))
    # اجرای granger برای هر جهت (statsmodels نیاز به آرایه با [target, cause])
    # جهت x -> y : قالب برای grangercausalitytests باید [y, x]
    maxlag_to_test = max(1, chosen_lag)
    res_xy = grangercausalitytests(df[['y', 'x']], maxlag=maxlag_to_test, verbose=False)
    pvals_xy = [res_xy[l][0]['ssr_ftest'][1] for l in range(1, maxlag_to_test+1)]
    best_p_xy = min(pvals_xy)
    x_causes_y = best_p_xy < significance

    res_yx = grangercausalitytests(df[['x', 'y']], maxlag=maxlag_to_test, verbose=False)
    pvals_yx = [res_yx[l][0]['ssr_ftest'][1] for l in range(1, maxlag_to_test+1)]
    best_p_yx = min(pvals_yx)
    y_causes_x = best_p_yx < significance

    return {
        'x_causes_y': {'best_pvalue': float(best_p_xy), 'chosen_lag': chosen_lag, 'significant': bool(x_causes_y)},
        'y_causes_x': {'best_pvalue': float(best_p_yx), 'chosen_lag': chosen_lag, 'significant': bool(y_causes_x)},
    }

# ---------- 2) MLP-based nonlinear test ----------
def mlp_predictive_test(x, y, lags=5, splits=5):
    """
    ساخت ویژگی: برای هر زمان t، ورودی شامل لَگ‌های x و لَگ‌های y (تا lags) است.
    مقایسه دو مدل:
      - model_base: فقط لَگ‌های target (مثلاً برای بررسی x->y, target=y -> use y lags)
      - model_full: لَگ‌های target + لَگ‌های candidate cause (x lags)
    با cross-validation زمانی (TimeSeriesSplit) و مقایسه MSE ها با t-test زوجی.
    برمی‌گرداند: (mse_base_mean, mse_full_mean, pvalue_paired_ttest)
    """
    # ساخت ماتریسِ تاخیری
    def build_matrix(a, b, lags):
        # a, b pandas Series, هم‌اندازه
        df = pd.concat([a, b], axis=1).dropna()
        df.columns = ['x', 'y']
        rows = []
        for t in range(lags, len(df)):
            row = {}
            # lagged x
            for L in range(1, lags+1):
                row[f'x_lag{L}'] = df['x'].iloc[t-L]
            # lagged y
            for L in range(1, lags+1):
                row[f'y_lag{L}'] = df['y'].iloc[t-L]
            row['y_target'] = df['y'].iloc[t]
            row['x_target'] = df['x'].iloc[t]
            rows.append(row)
        M = pd.DataFrame(rows).dropna()
        return M

    M = build_matrix(x, y, lags)
    if len(M) < splits+2:
        # داده کم؛ لگ را کم کن یا splits را کم کن
        return None

    X_base = M[[f'y_lag{L}' for L in range(1, lags+1)]].values
    X_full = M[[f'x_lag{L}' for L in range(1, lags+1)] + [f'y_lag{L}' for L in range(1, lags+1)]].values
    y_target = M['y_target'].values

    tscv = TimeSeriesSplit(n_splits=splits)
    mses_base = []
    mses_full = []

    for train_idx, test_idx in tscv.split(X_base):
        Xb_tr, Xb_te = X_base[train_idx], X_base[test_idx]
        Xf_tr, Xf_te = X_full[train_idx], X_full[test_idx]
        y_tr, y_te = y_target[train_idx], y_target[test_idx]

        mbase = MLPRegressor(hidden_layer_sizes=MLP_HIDDEN, max_iter=MLP_EPOCHS, random_state=0)
        mfull = MLPRegressor(hidden_layer_sizes=MLP_HIDDEN, max_iter=MLP_EPOCHS, random_state=0)

        mbase.fit(Xb_tr, y_tr)
        mfull.fit(Xf_tr, y_tr)

        pred_b = mbase.predict(Xb_te)
        pred_f = mfull.predict(Xf_te)

        mses_base.append(mean_squared_error(y_te, pred_b))
        mses_full.append(mean_squared_error(y_te, pred_f))

    # paired t-test: آیا MSE_full < MSE_base به طور معنی‌دار؟
    stat, pval = ttest_rel(mses_base, mses_full, alternative='greater') if hasattr(ttest_rel, 'alternative') else ttest_rel(mses_base, mses_full)
    # Note: scipy <1.9 ttest_rel does not support alternative param; we'll compare means if needed
    # fallback: compute p-two-sided and convert by direction
    if stat is None:
        # fallback compute two-sided
        stat, pval = ttest_rel(mses_base, mses_full)
        # if mean(mses_full) < mean(mses_base): we want one-sided p = pval/2
        if np.mean(mses_full) < np.mean(mses_base):
            pval = pval / 2.0
        else:
            pval = 1.0  # no improvement in desired direction

    return {
        'mse_base_mean': float(np.mean(mses_base)),
        'mse_full_mean': float(np.mean(mses_full)),
        'pvalue': float(pval),
        'mses_base': mses_base,
        'mses_full': mses_full,
        'improves': float(np.mean(mses_full)) < float(np.mean(mses_base))
    }

# ---------- Running the entire analysis ----------
def analyze_causality(price_a, price_b, name_a='A', name_b='B'):
    # price_a, price_b: pandas Series indexed by time
    # Convert to logarithmic efficiency
    rA = log_returns(price_a)
    rB = log_returns(price_b)

    # ایستا کردن
    rA_s, dA, pA = make_stationary(rA)
    rB_s, dB, pB = make_stationary(rB)

    print(f"Stationarity check: diffs applied: {name_a} d={dA}, adf_p={pA:.4f}; {name_b} d={dB}, adf_p={pB:.4f}")

    # هم‌اندازه کن و خطوط پاک‌سازی
    common_index = rA_s.index.intersection(rB_s.index)
    rA_s = rA_s.loc[common_index]
    rB_s = rB_s.loc[common_index]

    # 1) Granger test
    print("\n== Granger causality (linear) ==")
    gr = granger_test_matrix(rA_s, rB_s, maxlag=MAX_LAG, significance=SIGNIFICANCE)
    print(f"{name_a} -> {name_b}: pVal = {gr['x_causes_y']['best_pvalue']:.4f}, chosen_lag = {gr['x_causes_y']['chosen_lag']}, significant={gr['x_causes_y']['significant']}")
    print(f"{name_b} -> {name_a}: pVal = {gr['y_causes_x']['best_pvalue']:.4f}, chosen_lag = {gr['y_causes_x']['chosen_lag']}, significant={gr['y_causes_x']['significant']}")

    # 2) Nonlinear predictive test (A -> B)
    print("\n== Nonlinear predictive test (MLP) ==")
    LAGS = min(5, max(1, len(rA_s)//20))  # انتخاب معقول لَگ بر اساس طول داده
    print(f"Using lags = {LAGS} for MLP predictive tests (time-series CrossValidation={TS_SPLITS})")

    res_A_to_B = mlp_predictive_test(rA_s, rB_s, lags=LAGS, splits=min(TS_SPLITS, max(2, len(rA_s)//10)))
    res_B_to_A = mlp_predictive_test(rB_s, rA_s, lags=LAGS, splits=min(TS_SPLITS, max(2, len(rA_s)//10)))

    if res_A_to_B is None or res_B_to_A is None:
        print("Not enough data for MLP predictive test (increase samples or reduce lags/splits).")
    else:
        print(f"{name_a} -> {name_b}: mse_base={res_A_to_B['mse_base_mean']:.6f}, mse_full={res_A_to_B['mse_full_mean']:.6f}, pVal ={res_A_to_B['pvalue']:.4f}, improves={res_A_to_B['improves']}")
        print(f"{name_b} -> {name_a}: mse_base={res_B_to_A['mse_base_mean']:.6f}, mse_full={res_B_to_A['mse_full_mean']:.6f}, pVal ={res_B_to_A['pvalue']:.4f}, improves={res_B_to_A['improves']}")

    # تصمیم‌گیر ساده
    print("\n== Decision summary ==")
    # prefer nonlinear test if available, else rely on granger
    def decide(dir_lin, dir_nonlin):
        lin = dir_lin['significant']
        nonlin = (dir_nonlin is not None and dir_nonlin['improves'] and dir_nonlin['pvalue'] < SIGNIFICANCE)
        if nonlin and lin:
            return "Strong evidence (linear + nonlinear)"
        if nonlin and not lin:
            return "Evidence (nonlinear) — linear not significant"
        if lin and not nonlin:
            return "Evidence (linear) — nonlinear not significant"
        return "No evidence"

    dec_ab = decide(gr['x_causes_y'], res_A_to_B)
    dec_ba = decide(gr['y_causes_x'], res_B_to_A)
    print(f"{name_a} -> {name_b}: {dec_ab}")
    print(f"{name_b} -> {name_a}: {dec_ba}")

    # Compare the strength of the direction as well.
    score_ab = 0
    score_ba = 0
    if gr['x_causes_y']['significant']: score_ab += 1
    if gr['y_causes_x']['significant']: score_ba += 1
    if res_A_to_B is not None and res_A_to_B['improves']: score_ab += 1
    if res_B_to_A is not None and res_B_to_A['improves']: score_ba += 1

    if score_ab > score_ba:
        print(f"\nOverall: {name_a} is more likely to be the cause of {name_b} (score {score_ab} vs {score_ba}).")
    elif score_ba > score_ab:
        print(f"\nOverall: {name_b} is more likely to be the cause of {name_a} (score {score_ba} vs {score_ab}).")
    else:
        print("\nOverall: Equal or inconclusive evidence.")




In [20]:
df.columns

Index(['usd', 'gold', 'uae', 'usd return', 'gold return', 'uae return'], dtype='object')

In [21]:
print("base on raw price\nGold Vs USD")

cols = ['gold', 'uae']

analyze_causality(
    price_a=df['gold'], 
    price_b=df['usd'],
    name_a='gold', 
    name_b='usd')



base on raw price
Gold Vs USD
Stationarity check: diffs applied: gold d=0, adf_p=0.0000; usd d=0, adf_p=0.0000

== Granger causality (linear) ==
gold -> usd: pVal = 0.0004, chosen_lag = 1, significant=True
usd -> gold: pVal = 0.0013, chosen_lag = 1, significant=True

== Nonlinear predictive test (MLP) ==
Using lags = 5 for MLP predictive tests (time-series CrossValidation=5)
gold -> usd: mse_base=0.000597, mse_full=0.000764, pVal =0.3220, improves=False
usd -> gold: mse_base=0.000739, mse_full=0.000844, pVal =0.4238, improves=False

== Decision summary ==
gold -> usd: Evidence (linear) — nonlinear not significant
usd -> gold: Evidence (linear) — nonlinear not significant

Overall: Equal or inconclusive evidence.


In [22]:
print("base on raw price\nAED Vs USD")

analyze_causality(
    price_a=df['uae'], 
    price_b=df['usd'],
    name_a='uae', 
    name_b='usd')

base on raw price
AED Vs USD
Stationarity check: diffs applied: uae d=0, adf_p=0.0000; usd d=0, adf_p=0.0000

== Granger causality (linear) ==
uae -> usd: pVal = 0.0000, chosen_lag = 1, significant=True
usd -> uae: pVal = 0.0104, chosen_lag = 1, significant=True

== Nonlinear predictive test (MLP) ==
Using lags = 5 for MLP predictive tests (time-series CrossValidation=5)
uae -> usd: mse_base=0.000597, mse_full=0.000732, pVal =0.3872, improves=False
usd -> uae: mse_base=0.000566, mse_full=0.000691, pVal =0.4018, improves=False

== Decision summary ==
uae -> usd: Evidence (linear) — nonlinear not significant
usd -> uae: Evidence (linear) — nonlinear not significant

Overall: Equal or inconclusive evidence.


### Base on Return

In [23]:
print("base on Returns\nGold Vs USD")
analyze_causality(
    price_a=df['gold return'], 
    price_b=df['usd return'], 
    name_a='gold return', 
    name_b='usd return')


base on Returns
Gold Vs USD
Stationarity check: diffs applied: gold return d=0, adf_p=0.0000; usd return d=0, adf_p=0.0000

== Granger causality (linear) ==
gold return -> usd return: pVal = 0.0204, chosen_lag = 1, significant=True
usd return -> gold return: pVal = 0.4713, chosen_lag = 1, significant=False

== Nonlinear predictive test (MLP) ==
Using lags = 5 for MLP predictive tests (time-series CrossValidation=5)
gold return -> usd return: mse_base=2.165992, mse_full=3.020670, pVal =0.0930, improves=False
usd return -> gold return: mse_base=2.661767, mse_full=3.587915, pVal =0.0884, improves=False

== Decision summary ==
gold return -> usd return: Evidence (linear) — nonlinear not significant
usd return -> gold return: No evidence

Overall: gold return is more likely to be the cause of usd return (score 1 vs 0).


In [24]:
print("base on Returns\nAED Vs USD")
analyze_causality(
    price_a=df['uae return'], 
    price_b=df['usd return'], 
    name_a='uae return', 
    name_b='usd return')

base on Returns
AED Vs USD
Stationarity check: diffs applied: uae return d=0, adf_p=0.0000; usd return d=0, adf_p=0.0000

== Granger causality (linear) ==
uae return -> usd return: pVal = 0.1168, chosen_lag = 1, significant=False
usd return -> uae return: pVal = 0.0072, chosen_lag = 1, significant=True

== Nonlinear predictive test (MLP) ==
Using lags = 5 for MLP predictive tests (time-series CrossValidation=5)
uae return -> usd return: mse_base=2.700588, mse_full=2.749615, pVal =0.8376, improves=False
usd return -> uae return: mse_base=2.314151, mse_full=2.524580, pVal =0.6172, improves=False

== Decision summary ==
uae return -> usd return: No evidence
usd return -> uae return: Evidence (linear) — nonlinear not significant

Overall: usd return is more likely to be the cause of uae return (score 1 vs 0).
